# Imports

In [39]:
import pandas as pd
import os
import altair as alt

from tensorboard.backend.event_processing.event_accumulator import EventAccumulator

# Tensorboard

Launch a TensorBoard Session in VS Code or...

Run the below (you may have to run the second command twice)

In [ ]:
%load_ext tensorboard

In [ ]:
%tensorboard --logdir=log

or use

```
!tensorboard --logdir log
```
if the above doesn't work

# Training (and evaluation)

Use ```cifar_train.py``` to train on the imbalanced version of CIFAR 10. This is to get the results in Table 2.

Call it with the following flags/parameters:

* ```gpu```: ```0```
* ```imb_type```: ```exp``` (means that the class frequency decays exponentially)
* ```imb_factor```: ```0.01``` (the ratio of the lowest class frequency to the highest class frequency)
* ```loss_type```:
    * ```CE```: Cross Entropy Loss
    * ```LogAdj```: Logit Adjusted Loss
    * ```LDAM```: LDAM loss
* ```train_rule``` (how/when to weight the loss):
    * ```None```: default training. No weighting.
    * ```Reweight```: weight the per-class loss multiplicatively by inverse effective frequency. Also requires beta argument.
    * ```ClassWeight```: weight the per-class loss multiplicatively by inverse frequency, i.e. Reweight with $\beta = 1$
    * ```DRW```: delayed reweighting. Reweight the loss ONLY after 160th epoch. Also requires beta argument.
    * ```DRW_ClassWeight```. Reweight the loss ONLY after 160th epoch, with $\beta = 1$.
* ```beta```: value of $\beta$

The results can be found in Tensorboard: the 'Best' value corresponds to the final "test_top1_best" and the 'Final' value, to the final "test_val_top1" value.

## Experiments and Results

### Cao et. al

#### ERM

In [ ]:
!python cifar_train.py --gpu 0 --imb_type exp --imb_factor 0.01 --loss_type CE --train_rule None --exp_str "01"

#### LDAM

In [ ]:
!python cifar_train.py --gpu 0 --imb_type exp --imb_factor 0.01 --loss_type LDAM --train_rule None --exp_str "02"

#### LDAM+DRW

In [ ]:
!python cifar_train.py --gpu 0 --imb_type exp --imb_factor 0.01 --loss_type LDAM --train_rule DRW --beta 0.9999 --exp_str "03"

### Cui et. al

For $\beta = 0.9, 0.99, 0.999, 0.9999$

In [ ]:
!python cifar_train.py --gpu 0 --imb_type exp --imb_factor 0.01 --loss_type CE --train_rule Reweight --beta 0.9999 --exp_str "04"

In [ ]:
!python cifar_train.py --gpu 0 --imb_type exp --imb_factor 0.01 --loss_type CE --train_rule Reweight --beta 0.9990 --exp_str "05"

In [ ]:
!python cifar_train.py --gpu 0 --imb_type exp --imb_factor 0.01 --loss_type CE --train_rule Reweight --beta 0.9900 --exp_str "06"

In [ ]:
!python cifar_train.py --gpu 0 --imb_type exp --imb_factor 0.01 --loss_type CE --train_rule Reweight --beta 0.9000 --exp_str "07"

### Additional experiments

#### $\beta = 1$

In [ ]:
!python cifar_train.py --gpu 0 --imb_type exp --imb_factor 0.01 --loss_type CE --train_rule ClassWeight --exp_str "08"

#### LDAM+DRW ($\beta=1$)

In [ ]:
!python cifar_train.py --gpu 0 --imb_type exp --imb_factor 0.01 --loss_type LDAM --train_rule DRW_ClassWeight --exp_str "09"

#### Logit Adjustment

In [ ]:
!python cifar_train.py --gpu 0 --imb_type exp --imb_factor 0.01 --loss_type LogAdj --train_rule None --exp_str "10"

#### Logit Adjustment+DRW ($\beta = 1$)

In [ ]:
!python cifar_train.py --gpu 0 --imb_type exp --imb_factor 0.01 --loss_type LogAdj --train_rule DRW_ClassWeight --exp_str "11"

### Read the results

* For Table 2, Figures 7, 8 and 9
* Github Web UI won't render the Altair outputs. Make sure to clone the repo, checkout to this branch on your local clone and then you'll see the outputs.

In [40]:
def load_tensorboard_data(log_dir: str, print_scalars: bool = False) -> pd.DataFrame:
    """
    Returns the logs from Tensorboard SummaryWriter in blob storage as a pandas DataFrame.

    Args:
        log_dir (str): the path to the Tensorboard logs
        print_scalars (bool): whether or not to print the list of scalars from the EventAccumulator. By default False (so doesn't print).
    """
    event_acc = EventAccumulator(log_dir)
    event_acc.Reload()

    data = []
    scalars = event_acc.Tags()["scalars"]
    if print_scalars:
        print(scalars)

    for tag in scalars:
        events = event_acc.Scalars(tag)
        steps = [event.step for event in events]
        values = [event.value for event in events]
        for step, value in list(zip(steps, values)):
            data.append({"tag": tag, "step": step, "value": value})

    df = pd.DataFrame(data)
    return df

#### Overall metrics

In [41]:
# create a dictionary of dfs for each experiment

exp_dict = {}

In [42]:
# Here we collect the results from each experiment from Tensorboard logs

get_exp_no = lambda s: s.split("_")[
    -1
]  # function to remove the '_{exp_no}' from the folder name

_exp_names = sorted(
    os.listdir("./log"), key=get_exp_no
)  # the folder names - sorted by experiment no,
exp_names = [
    "_".join(_exp_name.split("_")[:-1])
    .replace("cifar10_resnet32_", "")
    .replace("_exp_0.01", "")
    for _exp_name in _exp_names
]
# the experiment names but stripped of redundant info (since all models are cifar10, resnet32 with exp imbalance type and 0.01 imbal ratio)

exp_nos = [
    get_exp_no(_exp_name) for _exp_name in _exp_names
]  # the experiment numbers that have logs

for exp_no, _exp_name, exp_name in list(zip(exp_nos, _exp_names, exp_names)):
    exp_dict[int(exp_no)] = load_tensorboard_data(
        log_dir="./log/" + _exp_name
    )  # get the logs df for each experiment
    exp_dict[int(exp_no)] = (
        exp_dict[int(exp_no)].assign(exp_no=exp_no).assign(exp_name=exp_name)
    )  # include a column for the experiment number and name

In [43]:
# combine all the dfs

whole_df = pd.concat([df for _, df in exp_dict.items()])

##### Table 2

In [44]:
(
    whole_df[whole_df.tag == "acc/test_val_top1"]
    .sort_values(["exp_no", "step"])
    .groupby(["exp_no", "exp_name"])
    .agg({"value": ["last", "max"]})
    .reset_index()
    .drop(["exp_no"], axis=1)
)

/tmp/ipykernel_17215/2459142924.py:2: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  whole_df[whole_df.tag == "acc/test_val_top1"]


exp_name      value           
                                 last        max
0                  CE_None  71.619995  72.019997
1                LDAM_None  73.369995  73.830002
2          LDAM_DRW_0.9999  78.099998  78.209999
3       CE_Reweight_0.9999  72.559998  72.759995
4        CE_Reweight_0.999  72.769997  73.040001
5         CE_Reweight_0.99  71.680000  72.269997
6          CE_Reweight_0.9  70.650002  70.820000
7           CE_ClassWeight  72.479996  72.949997
8     LDAM_DRW_ClassWeight  77.180000  77.939995
9              LogAdj_None  77.449997  77.739998
10  LogAdj_DRW_ClassWeight  79.540001  79.919998

##### Figure 9

In [45]:
# plot different metrics across the experiments, using Altair

exp_selector = alt.selection_point(fields=["exp_name"], bind="legend")

for tag in ["loss/train", "acc/train_top1", "acc/test_val_top1"]:
    (
        alt.Chart(data=whole_df[whole_df.tag == tag])
        .mark_line()
        .encode(
            x="step",
            y=alt.Y("value", title=tag),
            color=alt.Color("exp_name"),
            opacity=alt.condition(exp_selector, alt.value(1), alt.value(0)),
        )
        .add_params(exp_selector)
        .properties(width=800, title=tag)
    ).display()

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

#### Figures 7 and 8: Per class accuracies

In [46]:
# create a dictionary of dfs for each experiment

exp_dict = {}

In [47]:
# Here we collect the results from each experiment from Tensorboard logs

get_exp_no = lambda s: s.split("_")[
    -1
]  # function to remove the '_{exp_no}' from the folder name

_exp_names = sorted(
    os.listdir("./log"), key=get_exp_no
)  # the folder names - sorted by experiment no,
exp_names = [
    "_".join(_exp_name.split("_")[:-1])
    .replace("cifar10_resnet32_", "")
    .replace("_exp_0.01", "")
    for _exp_name in _exp_names
]
# the experiment names but stripped of redundant info (since all models are cifar10, resnet32 with exp imbalance type and 0.01 imbal ratio)

exp_nos = [
    get_exp_no(_exp_name) for _exp_name in _exp_names
]  # the experiment numbers that have logs

for exp_no, _exp_name, exp_name in list(zip(exp_nos, _exp_names, exp_names)):
    exp_dict[int(exp_no)] = pd.DataFrame()
    for class_num in range(10):
        exp_dict[int(exp_no)] = pd.concat(
            [
                exp_dict[int(exp_no)],
                load_tensorboard_data(
                    log_dir="./log/"
                    + _exp_name
                    + "/acc/test_val_cls_acc/"
                    + str(class_num)
                ).assign(class_num=class_num + 1),
            ]
        )  # get the logs df for each experiment
        exp_dict[int(exp_no)] = (
            exp_dict[int(exp_no)].assign(exp_no=exp_no).assign(exp_name=exp_name)
        )  # include a column for the experiment number and name and class

In [48]:
# combine all the dfs

whole_df = pd.concat([df for _, df in exp_dict.items()])

In [49]:
# plot final test accuracy

exp_selector = alt.selection_point(fields=["exp_name"], toggle="true")

tag = "acc/test_val_cls_acc"

exp_chart = (
    alt.Chart(data=whole_df[["exp_name"]].drop_duplicates())
    .mark_circle(filled=True, size=200)
    .encode(
        y="exp_name:N",
        color="exp_name:N",
        opacity=alt.condition(exp_selector, alt.value(1), alt.value(0.2)),
    )
    .add_params(exp_selector)
)

chart = (
    alt.Chart(data=whole_df[(whole_df.tag == tag) & (whole_df.step == 199)])
    .mark_bar()
    .encode(
        x=alt.X("exp_no:N", title=None, axis=alt.Axis(ticks=False, labels=False)),
        column=alt.Column("class_num:O"),
        y=alt.Y("value", title=tag),
        color=alt.Color("exp_name:N", legend=None),
    )
    .transform_filter(exp_selector)
    .add_params(exp_selector)
)

exp_chart | chart

alt.HConcatChart(...)